# Version 1 of the final project 

## Preprocessing 

In [6]:
#Imports necessary libraries 
import pandas as pd
import numpy as np
import networkx as nx
import json as js

In [16]:
data1 = js.load(open("myresponse1.json")) #Reads in the json file 
data2 = js.load(open("myresponse2.json"))
df1 = pd.DataFrame(data1['data']) #Writes it to a dataframe 
df2 = pd.DataFrame(data2['data'])
result = pd.concat([df1,df2]) #Concatenates the data together 

## Numerical Analysis  